### Predict Sahel rainfall with Echo State Networks

In this project we work with **C**limate **I**ndex **C**ollection based on **Mo**del **D**ata (CICMoD) data set (https://github.com/MarcoLandtHayen/climate_index_collection). 

Here, we will try to **predict future** Sahel rainfall (lead times 1 / 3 / 6 months) from current and past information (t<=0) of all input features (including PREC_SAHEL) with **ESN** models:

- Prepare inputs and targets.
- Set up model.
- Evaluate model performance.

**Note:** We start with predicting future Sahel rainfall from its own history alone, hence with **univariate** inputs. Then, we add further input features to have **multivariate** inputs. And ultimately, we add **months as additional input features**.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from json import dump, load
from pathlib import Path

### Import additional functions:
from predict_sahel_rainfall.plot import bar_color
from predict_sahel_rainfall.preprocessing import prepare_inputs_and_target
from predict_sahel_rainfall.ESN_functions import ESN, setESN, trainESN, predESN

### Prepare inputs and targets: Univariate

Load collection of climate indices directly from GitHub release.
Use the complete preprocessing pipeline function.
**Note:** Don't need validation data, only need training and test data. Hence, set ```train_val_split = 1.0```.

In [28]:
## Set common parameters (except ESN and lead time) for data preprocessing:

# Set url to csv file containing CICMoD indices from desired release:
data_url = (
    "https://github.com/MarcoLandtHayen/climate_index_collection/"
    "releases/download/v2023.03.29.1/climate_indices.csv"
)

# Select target index:
target_index = 'PREC_SAHEL'

# Select all input features:
input_features = [
    'AMO', 'ENSO_12', 'ENSO_3', 'ENSO_34', 'ENSO_4', 'NAO_PC', 'NAO_ST', 
    'NP', 'PDO_PC', 'PREC_SAHEL', 'SAM_PC', 'SAM_ZM', 'SAT_N_ALL', 'SAT_N_LAND',
    'SAT_N_OCEAN', 'SAT_S_ALL', 'SAT_S_LAND', 'SAT_S_OCEAN', 'SOI',
    'SSS_ENA', 'SSS_NA', 'SSS_SA', 'SSS_WNA', 'SST_ESIO', 'SST_HMDR',
    'SST_MED', 'SST_TNA', 'SST_TSA', 'SST_WSIO'
]

# # Select subset of input features:
# input_features = [
#     'PREC_SAHEL',
# ]

# Choose, whether to add months as one-hot encoded features:
add_months = True

# Choose, whether to normalize target index:
norm_target = True

# Specify input length:
input_length = 24

# Specify amount of combined training and validation data relative to test data:
train_test_split = 0.9

# Specify relative amount of combined training and validation used for training:
train_val_split = 1.0

## Optionally choose to scale or normalize input features according to statistics from training data:
# 'no': Keep raw input features.
# 'scale_01': Scale input features with min/max scaling to [0,1].
# 'scale_11': Scale input features with min/max scaling to [-1,1].
# 'norm': Normalize input features, hence subtract mean and divide by std dev.
scale_norm = 'scale_11'

In [29]:
# Set parameters for ESN model:
verbose = False
n_layers = 1 # Number of ESN layers in the model.
n_res = 300 # Number of reservoir units.
W_in_lim = 0.1 # Initialize input weights from random uniform distribution in [- W_in_lim, + W_in_lim]
leak_rate = 0.05 # Leak rate used in transition function of reservoir states.
leak_rate_first_step_YN = True # If true, multiply with alpha already in calculating first timestes's res. states.
leaky_integration_YN = True # If True, multiply previous time steps' reservoir states with (1-a).
                            # If False, omit multiplication with (1-a) in reservoir state transition. 
                            # But in any case still multiply new time steps' input (and reservoir recurrence) 
                            # with leakrate a after activation.
activation = 'tanh' # Desired activation function to be used in calculating reservoir state transition.
spec_radius = 0.8 # Spectral radius, becomes largest Eigenvalue of reservoir weight matrix.
sparsity = 0.3 # Sparsity of reservoir weight matrix.
out_features = 1 # Single target value.

In [30]:
# Set choice of ESMs:
ESMs = ['CESM', 'FOCI']

# Set choice of lead times:
lead_times = [1,3,6]

# Set number of runs per setting:
n_runs = 3

# Get number of input features, depending on whether or not months are added as additional features:
if add_months:
    n_features = len(input_features) + 12
else:
    n_features = len(input_features)

# Check number of input channels and input length:
print('Number of input features:',n_features)

Number of input features: 41


In [37]:
# Set parameters for test:
input_length = 24
n_features = 10
verbose = False
n_layers = 1
n_res = 300
W_in_lim = 0.1
leak_rate = 0.05
leak_rate_first_step_YN = True
leaky_integration_YN = True
activation = 'tanh'
spec_radius = 0.8
sparsity = 0.3
out_features = 1


model, model_short, all_states = setESN(
    input_length=input_length, 
    in_features=n_features,                                        
    out_features=out_features, 
    n_layers=n_layers,                                        
    n_res=n_res, 
    W_in_lim=W_in_lim, 
    leak_rate=leak_rate,
    leak_rate_first_step_YN=leak_rate_first_step_YN,
    leaky_integration_YN = leaky_integration_YN,
    activation=activation, 
    spec_radius=spec_radius,
    sparsity=sparsity, 
    verbose=verbose)

In [31]:
## Initializs storages for loss curves and correlation, dimension (#ESMs, #lead times, #runs).
train_loss_all = np.zeros((len(ESMs),len(lead_times),n_runs))
test_loss_all = np.zeros((len(ESMs),len(lead_times),n_runs))
train_correl_all = np.zeros((len(ESMs),len(lead_times),n_runs))
test_correl_all = np.zeros((len(ESMs),len(lead_times),n_runs))

## Loop over ESMs:
for m in range(len(ESMs)):
    
    # Get current ESM:
    ESM = ESMs[m]
    
    # Print status:
    print('ESM:',m+1,'of',len(ESMs))

    ## Loop over lead times:
    for l in range(len(lead_times)):
        
        # Get current lead time:
        lead_time = lead_times[l]
        
        # Print status:
        print('  lead time:',l+1,'of',len(lead_times))

        # Prepare inputs and target for current ESM and lead time:
        (
            train_input,
            train_target,
            _,
            _,
            test_input,
            test_target,
            train_mean,
            train_std,
            train_min,
            train_max,
        ) = prepare_inputs_and_target(    
            data_url=data_url,
            ESM=ESM,
            target_index=target_index,
            input_features=input_features,
            add_months=add_months,
            norm_target=norm_target,
            lead_time=lead_time,
            input_length=input_length,
            train_test_split=train_test_split,
            train_val_split=train_val_split,
            scale_norm=scale_norm,
        )
        
        # Add dummy column of ONEs as first input time step:
        train_input = np.concatenate([np.ones((train_input.shape[0],1,train_input.shape[2])), train_input], axis=1)
        test_input = np.concatenate([np.ones((test_input.shape[0],1,test_input.shape[2])), test_input], axis=1)

        # Loop over desired number of training runs:
        for r in range(n_runs):
            
            # Print status:
            print('    run:',r+1,'of',n_runs)
            
            ## Set up ESN model:
            # Get complete model (output = target prediction) plus short model (output final reservoir states from all layers)
            # and all_states (= another shortened model that gives reservoir states for ALL time steps for all inputs).
            # Manually add dummy column to input length:
            model, model_short, all_states = setESN(
                input_length=input_length+1, 
                in_features=n_features,                                        
                out_features=out_features, 
                n_layers=n_layers,                                        
                n_res=n_res, 
                W_in_lim=W_in_lim, 
                leak_rate=leak_rate,
                leak_rate_first_step_YN=leak_rate_first_step_YN,
                leaky_integration_YN = leaky_integration_YN,
                activation=activation, 
                spec_radius=spec_radius,
                sparsity=sparsity, 
                verbose=verbose)
            
            # Train ESN model's output weights and bias
            model = trainESN(model, model_short, train_input, train_target, verbose=verbose)

            # Get predictions from trained ESN model and evaluation metrics on model performance:
            (
                train_pred, 
                test_pred, 
                _, 
                _, 
                _, 
                _
            ) = predESN(model, train_input, test_input, train_target, test_target, verbose=verbose)
            
            # Compute mse of model predictions vs. true targets:
            train_loss = np.mean((train_target-train_pred)**2)
            test_loss = np.mean((test_target-test_pred)**2)

            # Compute correlation coefficient of model predictions vs. true targets:
            train_correl = np.corrcoef(np.stack([train_target[:,0],train_pred[:,0]]))[0,1]
            test_correl = np.corrcoef(np.stack([test_target[:,0],test_pred[:,0]]))[0,1]
            
            # Store results:
            train_loss_all[m,l,r] = train_loss
            test_loss_all[m,l,r] = test_loss
            train_correl_all[m,l,r] = train_correl
            test_correl_all[m,l,r] = test_correl       

ESM: 1 of 2
  lead time: 1 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3
  lead time: 2 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3
  lead time: 3 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3
ESM: 2 of 2
  lead time: 1 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3
  lead time: 2 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3
  lead time: 3 of 3
    run: 1 of 3
    run: 2 of 3
    run: 3 of 3


In [33]:
### Store results:

# ## ESN - multivariate:

# # Specify model setup:
# setup = 'ESN - multivariate'

# # Save loss and correlation results:
# np.save('../results/quickrun_ESN_multivariate_train_loss_all.npy', train_loss_all)
# np.save('../results/quickrun_ESN_multivariate_test_loss_all.npy', test_loss_all)
# np.save('../results/quickrun_ESN_multivariate_train_correl_all.npy', train_correl_all)
# np.save('../results/quickrun_ESN_multivariate_test_correl_all.npy', test_correl_all)

# # Store parameters:
# parameters = {
#     "setup": setup,
#     "data_url": data_url,
#     "target_index": target_index,
#     "input_features": input_features,
#     "add_months": add_months,
#     "norm_target": norm_target,
#     "input_length": input_length,
#     "train_test_split": train_test_split,
#     "train_val_split": train_val_split,
#     "train_val_split": train_val_split,
#     "scale_norm": scale_norm,  
#     "verbose": verbose,  
#     "n_layers": n_layers,  
#     "n_res": n_res,  
#     "W_in_lim": W_in_lim,  
#     "leak_rate": leak_rate,  
#     "leak_rate_first_step_YN": leak_rate_first_step_YN,  
#     "leaky_integration_YN": leaky_integration_YN,  
#     "activation": activation,  
#     "spec_radius": spec_radius,  
#     "sparsity": sparsity,  
#     "out_features": out_features,  
#     "ESMs": ESMs,
#     "lead_times": lead_times,
#     "n_runs": n_runs,   
# }

# path_to_store_results = Path('../results')
# with open(path_to_store_results / "quickrun_ESN_multivariate_parameters.json", "w") as f:
#     dump(parameters, f)

# #######################################
    
## ESN - multivariate - months as additional input features:

# Specify model setup:
setup = 'ESN - multivariate - with months'

# Save loss and correlation results:
np.save('../results/quickrun_ESN_multivariate_with_months_train_loss_all.npy', train_loss_all)
np.save('../results/quickrun_ESN_multivariate_with_months_test_loss_all.npy', test_loss_all)
np.save('../results/quickrun_ESN_multivariate_with_months_train_correl_all.npy', train_correl_all)
np.save('../results/quickrun_ESN_multivariate_with_months_test_correl_all.npy', test_correl_all)

# Store parameters:
parameters = {
    "setup": setup,
    "data_url": data_url,
    "target_index": target_index,
    "input_features": input_features,
    "add_months": add_months,
    "norm_target": norm_target,
    "input_length": input_length,
    "train_test_split": train_test_split,
    "train_val_split": train_val_split,
    "train_val_split": train_val_split,
    "scale_norm": scale_norm,  
    "verbose": verbose,  
    "n_layers": n_layers,  
    "n_res": n_res,  
    "W_in_lim": W_in_lim,  
    "leak_rate": leak_rate,  
    "leak_rate_first_step_YN": leak_rate_first_step_YN,  
    "leaky_integration_YN": leaky_integration_YN,  
    "activation": activation,  
    "spec_radius": spec_radius,  
    "sparsity": sparsity,  
    "out_features": out_features,  
    "ESMs": ESMs,
    "lead_times": lead_times,
    "n_runs": n_runs,   
}

path_to_store_results = Path('../results')
with open(path_to_store_results / "quickrun_ESN_multivariate_with_months_parameters.json", "w") as f:
    dump(parameters, f)

In [7]:
# ### Reload results:
   
# ## ESN - multivariate:

# # Load loss and correlation results:
# train_loss_all = np.load('../results/quickrun_ESN_multivariate_train_loss_all.npy')
# test_loss_all = np.load('../results/quickrun_ESN_multivariate_test_loss_all.npy',)
# train_correl_all = np.load('../results/quickrun_ESN_multivariate_train_correl_all.npy')
# test_correl_all = np.load('../results/quickrun_ESN_multivariate_test_correl_all.npy')

# # Load parameters:
# path_to_store_results = Path('../results')
# with open(path_to_store_results / 'quickrun_ESN_multivariate_parameters.json', 'r') as f:
#     parameters=load(f)

# ESMs = parameters['ESMs']
# lead_times = parameters['lead_times']
# n_runs = parameters['n_runs']

# #######################################
    
# ## ESN - multivariate - with months:

# # Load loss and correlation results:
# train_loss_all = np.load('../results/quickrun_ESN_multivariate_with_months_train_loss_all.npy')
# test_loss_all = np.load('../results/quickrun_ESN_multivariate_with_months_test_loss_all.npy',)
# train_correl_all = np.load('../results/quickrun_ESN_multivariate_with_months_train_correl_all.npy')
# test_correl_all = np.load('../results/quickrun_ESN_multivariate_with_months_test_correl_all.npy')

# # Load parameters:
# path_to_store_results = Path('../results')
# with open(path_to_store_results / 'quickrun_ESN_multivariate_with_months_parameters.json', 'r') as f:
#     parameters=load(f)

# ESMs = parameters['ESMs']
# lead_times = parameters['lead_times']
# n_runs = parameters['n_runs']

### Postprocessing

We now have loss ('mse') and correlation for complete training and test data.

Next, we compute the **mean loss and correlation on test data over all runs**, separately for each ESM and lead time.

In [34]:
## Initializs storages for mean test loss and correlation, averaged over all training runs,
## dimension (#ESMs, #lead times).
test_loss_mean = np.zeros((len(ESMs),len(lead_times)))
test_correl_mean = np.zeros((len(ESMs),len(lead_times)))

## Loop over ESMs:
for m in range(len(ESMs)):
    
    ## Loop over lead times:
    for l in range(len(lead_times)):        
            
        # Get mean test loss and correlation over all training runs, for current ESM and lead time:
        test_loss_mean[m,l] = np.mean(test_loss_all[m,l])
        test_correl_mean[m,l] = np.mean(test_correl_all[m,l])

### Results: Multivariate ESN (without months as additional input features)

In [25]:
test_loss_mean

array([[1.08662774, 1.07734265, 1.09522613],
       [0.84196932, 0.8732742 , 0.88330637]])

In [26]:
test_correl_mean

array([[0.20153117, 0.20912285, 0.16223111],
       [0.15897595, 0.08291692, 0.0533908 ]])

### Results: Multivariate ESN (with months as additional input features)

In [35]:
test_loss_mean

array([[1.04180678, 1.06451105, 1.11768725],
       [0.83749761, 0.85866585, 0.85737492]])

In [36]:
test_correl_mean

array([[0.2557714 , 0.22284854, 0.15976369],
       [0.1840896 , 0.10410937, 0.07351633]])

### Discussion: ESN models - multivariate (with/without months as additional input features)

Here, we tried to **predict future** Sahel rainfall (lead times 1 / 3 / 6 months) from current and past information (t<=0) of all input features (including PREC_SAHEL) with **ESN** models.

We skipped predicting future Sahel rainfall from its own history alone, hence with **univariate** inputs.
Instead, we directly started with adding further input features to have **multivariate** inputs.
And then, we added **months as additional input features**, which gives us the slightly improved results.

However, we find better results for models trained on **CESM** data, compared to **FOCI**.
And we see unreasonable behaviour, e.g., for models trained on CESM data with lead time 6: Lowest loss is found for models trained on multivariate inputs and slightly higher loss for models trained on multivariate inputs including months. Would have expected the reverse order.